# Data Science Homework 4
Try different imbalanced classification datasets using $k$-fold cross validation and various classification methods.
TODO List:
- Make sure we can open all the data as either DataFrame or nparray
- Handle categorical data (tokenize, one-hot encoding, ....)
- Split each dataset into training and testing dataset.
- Perform any necessary sampling, imputaiton, encoding techniques depending on dataset
- Perform 5-fold cross-validation to select datasets.

In [1]:
import os
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
import category_encoders as ce # sklearn library
import xgboost

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn import preprocessing

DATA_DIR = './data'
files = os.listdir(DATA_DIR)
files.remove('.ipynb_checkpoints')

special_delims = { 'arcene_train.data': ' '}
print(files)

['covtype.data', 'dataset', 'biodeg.csv', 'eeg_eye_state.arff', 'heart_2020_cleaned.csv', 'phpAmSP4g.arff', 'WA_Fn-UseC_-Telco-Customer-Churn.csv', 'WineQT.csv', 'HTRU_2.csv', 'spambase.data', 'Raisin_Dataset.arff', 'UCI_Credit_Card.csv', 'income_evaluation.csv', 'abalone.data', 'arcene_train.data']


## Load datasets
Some datasets are in csv format, others have just the data. First convert to `DataFrame`s to allow for numeric, categorical values.

In [2]:

def load(name, header=True):
    sep = special_delims[name] if name in special_delims else ','
    name = os.path.join(DATA_DIR, name)
    if header:
        df = pd.read_csv(name, sep=sep)
    else:
        df = pd.read_csv(name, header=None, sep=sep)
    return df

## Encode Categorical Values

In [3]:
def encode_categorical(data:pd.DataFrame, method='ordinal'):
    if method == 'ordinal':
        encoder = ce.OrdinalEncoder()
    elif method == 'onehot':
        encoder = ce.OneHotEncoder()
    return encoder.fit_transform(data)

def scale_features(data:pd.DataFrame, method='standard') -> pd.DataFrame:
    if method == 'standard':
        scaler = preprocessing.StandardScaler()
    elif method == 'minmax':
        scaler = preprocessing.MinMaxScaler()
    return scaler.fit_transform(data)

## Train model:
The evalutation should be fixed on 5-fold cross validation, choose from `RandomForest`, `GBDT`, `XGBoost`,`LightBGM`, `CatBoost`, `KNN`, `Logistic Regression`,`MLP`, `SVM`.

Train a new model every iteration of the cross-validation.

In [ ]:

def get_model(method):
    if method == 'xgboost':
        model = xgboost.XGBClassifier()
    elif method == 'knn':
        model = KNeighborsClassifier()
    elif method == 'forest':
        model = RandomForestClassifier(10)
    else:
        print(f'{method} not supported.')
        model = None
    return model

def get_metric(method):
    if method == 'acc':
        met = metrics.accuracy_score
    elif method == 'auc':
        met = metrics.auc
    elif method == 'roc_auc':
        met = metrics.roc_auc_score
    elif method == 'f1':
        met = metrics.f1_score
    return met

def train(x, y, method='xgboost', metric='acc'):
    # Perform K-Fold cross validation
    metric = get_metric(metric)
    n_items = x.shape[0]
    kf = KFold( n_splits=5, shuffle=True)
    
    mean_score = 0 
    for train_index, test_index in kf.split(x):
        model = get_model(method)
        x_train, x_test = x.iloc[train_index,: ], x.iloc[test_index, :]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        y_train -= 1 # Class labels should be zero-based
        model.fit(x_train, y_train)
        
        # Evaluate
        preds = model.predict(x_test)
        score = metric(y_test, preds, average='macro')
        mean_score += score
        print(f'\t{mean_score}')
    print(f'Mean score over validation: {score} using {method}')
    return mean_score / i
        

## P1: How does feature scaling (e.g. performing normalization) affect performance?
With standardization we apply the formula
$$x' = \frac{x - \mu}{\sigma}$$
so that we have 0 mean in the training data.

For One-hot Encoding we should not use this with tree-based models. For these models we can use label encoding, feature-hashing.
We should first encode the categorical values, then train the models on all our datasets both with feature scaling and without.

In [11]:
headers = {'covtype.data': False}
for file in files:
    df = load(file, headers[file])
    
    # Encode categorical columns
    cats = df.select_dtypes(include=['object'])
    df[cats.columns] = encode_categorical(cats)
    
    control = df.copy() # Compare with standardized dataset

    df, labels = df.iloc[:, :-1], df.iloc[:, -1]
    control, c_labels = control.iloc[:, :-1], control.iloc[:, -1]
    # Feature scaling
    df = pd.DataFrame(scale_features(df))
    
    # Split Test, Training data
    #x_train, x_test, y_train, y_test = train_test_split(df, labels, test_size=0.3)
    #cx_train, cx_test, cy_train, cy_test, = train_test_split(control, c_labels, test_size=0.3)
    
    # Train model
    print(f'Training on {file}')
    #train(df, labels, method='forest', metric='f1')
    
    train(control, c_labels, method='forest', metric='f1')


Training on covtype.data
<class 'pandas.core.frame.DataFrame'> [0 1 2 3 5]
	0.007272842266468762
<class 'pandas.core.frame.DataFrame'> [0 1 2 3 4]
	0.014355271529569872
<class 'pandas.core.frame.DataFrame'> [0 1 3 4 5]
	0.021966093405106593
<class 'pandas.core.frame.DataFrame'> [2 4 5 6 8]
	0.02890087348480883
<class 'pandas.core.frame.DataFrame'> [0 1 2 3 4]
	0.03658530365373009
Mean score over validation: 0.007684430168921259 using forest


NameError: name 'i' is not defined